# LLM-powered AI Agents

Table of contents
1. Understanding LLMs
2. Tools
3. Chat-based AI Agents
4. Service-based AI agents

In [1]:
from language_models.proxy_client import BTPProxyClient
from language_models.settings import settings

proxy_client = BTPProxyClient(
    client_id=settings.CLIENT_ID,
    client_secret=settings.CLIENT_SECRET,
    auth_url=settings.AUTH_URL,
    api_base=settings.API_BASE,
)

## 1. Understanding LLMs

In [2]:
from language_models.models.llm import OpenAILanguageModel, ChatMessage, ChatMessageRole

In [3]:
llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model="gpt-35-turbo",
    max_tokens=256,
    temperature=0.0,
)

In [4]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!"""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Sentiment: Positive


In [5]:
prompt = """Take the following movie review and determine the sentiment of the review.

Movie review:
Wow! This movie was incredible. The acting was superb, and
the plot kept me on the edge of my seat. I highly recommend it!

Respond with positive or negative."""

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

positive


In [6]:
system_prompt = "Take the following movie review and determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Wow! This movie was incredible. The acting was superb, and the plot kept me on the edge of my seat. I highly recommend it!"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

1


In [7]:
system_prompt = "Take the following movie review determine the sentiment of the review. Respond with 1 (positive) or 0 (negative)."

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

I'm sorry, I am an AI language model and I do not have access to real-time weather information. I recommend checking a reliable weather website or using a weather app to get the most accurate and up-to-date forecast for Seattle.


In [8]:
system_prompt = """Take the following movie review and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a movie review, respond with -1."""

prompt = "Will it rain in Seattle today?"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
print(response)

-1


## 2. Tools

In [9]:
import json
from language_models.tools.tool import Tool
from pydantic import BaseModel, Field
from typing import Any

In [10]:
prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12" # answer: $1,289.98

response = llm.get_completion([ChatMessage(role=ChatMessageRole.USER, content=prompt)])
print(response)

Total Raw Cost = $1,290.98


In [11]:
def calculator(expression: str) -> Any:
    return eval(expression)

class Calculator(BaseModel):
    expression: str = Field(description="A math expression.")

calculator_tool = Tool(
    func=calculator,
    name="Calculator",
    description="Use this tool when you want to do calculations.",
    args_schema=Calculator
)
print(calculator_tool)

tool name: Calculator, tool description: Use this tool when you want to do calculations., tool input: {{'expression': {{'description': 'A math expression.', 'title': 'Expression', 'type': 'string'}}}}


In [12]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "To calculate the total raw cost, you need to add up all the individual costs.",
    "tool": "Calculator",
    "tool_input": {
        "expression": "549.72 + 6.98 + 41.00 + 35.00 + 552.00 + 76.16 + 29.12"
    }
}


In [13]:
print(calculator(**response["tool_input"]))

1289.98


In [14]:
system_prompt = f"""Take the following prompt and calculate the result.

Respond to the user as helpfully and accurately as possible.

You have access to the following tools: {calculator_tool}

Use a JSON blob to specify a thought, a tool by providing an tool key (tool name) and a tool_input key (tool input).

Valid "tool" values: {calculator_tool.name}

Always use the following JSON format:
{{
  "thought": "You should always think about what to do consider previous and subsequent steps",
  "tool": "The tool to use",
  "tool_input": "Valid key value pairs",
}}

Observation: tool result
... (this Thought/Tool/Observation can repeat N times)

When you know the answer, use the following JSON format:
{{
  "thought": "I now know what to respond",
  "tool": "Final Answer",
  "tool_input": "The final answer to the question",
}}"""

prompt = "Total Raw Cost = $549.72 + $6.98 + $41.00 + $35.00 + $552.00 + $76.16 + $29.12"

response = llm.get_completion([
    ChatMessage(role=ChatMessageRole.SYSTEM, content=system_prompt), 
    ChatMessage(role=ChatMessageRole.USER, content=prompt),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=json.dumps(response)),
    ChatMessage(role=ChatMessageRole.ASSISTANT, content=f"Response of Calculator tool: {calculator(**response['tool_input'])}"),
])
response = json.loads(response, strict=False)
print(json.dumps(response, indent=4))

{
    "thought": "I now know the total raw cost.",
    "tool": "Final Answer",
    "tool_input": "The total raw cost is $1289.98."
}


## 3. Chat-based AI Agents

In [15]:
from language_models.agents.react import ReActAgent
from language_models.tools.earthquake import earthquake_tools
from language_models.tools.current_date import current_date_tool

### Earthquake

In [16]:
system_prompt = """You are an United States Geological Survey expert who can answer questions regarding earthquakes and can run forecasts.

Use the current date tool to access the local date and time before using other tools.

Take the following question and answer it as accurately as possible.
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4-32k',
    max_tokens=2048,
    float=0.0,
)

class Output(BaseModel):
    content: str = Field(description="The final answer.")

earthquake_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="{question}",
    task_prompt_variables=["question"],
    tools=earthquake_tools + [current_date_tool],
    output_format=Output,
    iterations=10,
)

In [17]:
response = earthquake_agent.invoke({"question": "How many earthquakes have occurred for the past week with a magnitude of 5 or greater?"})

05/05/24 18:26:23 INFO Prompt:
How many earthquakes have occurred for the past week with a magnitude of 5 or greater?
05/05/24 18:26:29 INFO Raw response:
{
  "thought": "First, I need to get the current date to calculate the start date for the past week.",
  "tool": "Current Date",
  "tool_input": {}
}
05/05/24 18:26:29 INFO Thought:
First, I need to get the current date to calculate the start date for the past week.
05/05/24 18:26:29 INFO Tool:
Current Date
05/05/24 18:26:29 INFO Tool input:
{}
05/05/24 18:26:29 INFO Tool response:
2024-05-05 18:26:29.733633
05/05/24 18:26:39 INFO Raw response:
{
  "thought": "I need to count the number of earthquakes that have occurred in the past week with a magnitude of 5 or greater.",
  "tool": "Count",
  "tool_input": {
    "start_time": "2024-04-28T18:26:29.733633",
    "min_magnitude": 5
  }
}
05/05/24 18:26:39 INFO Thought:
I need to count the number of earthquakes that have occurred in the past week with a magnitude of 5 or greater.
05/05/24

In [18]:
print(response.final_answer["content"])

There have been 26 earthquakes in the past week with a magnitude of 5 or greater.


In [19]:
response = earthquake_agent.invoke({"question": "Query 10 earthquakes that occurred yesterday and have a magnitude > 3."})

05/05/24 18:26:43 INFO Prompt:
Query 10 earthquakes that occurred yesterday and have a magnitude > 3.
05/05/24 18:26:46 INFO Raw response:
Tool response:
2024-05-05 18:26:29.733633
05/05/24 18:26:48 INFO Raw response:
Tool response:
2024-05-06 18:26:29.733633
05/05/24 18:26:51 INFO Raw response:
{
  "thought": "I need to get the current date to calculate the date for 'yesterday'.",
  "tool": "Current Date",
  "tool_input": {}
}
05/05/24 18:26:51 INFO Thought:
I need to get the current date to calculate the date for 'yesterday'.
05/05/24 18:26:51 INFO Tool:
Current Date
05/05/24 18:26:51 INFO Tool input:
{}
05/05/24 18:26:51 INFO Tool response:
2024-05-05 18:26:51.577497
05/05/24 18:26:56 INFO Raw response:
{
  "thought": "I will now use the Query tool to find 10 earthquakes that occurred yesterday and have a magnitude greater than 3.",
  "tool": "Query",
  "tool_input": {
    "start_time": "2024-05-04T00:00:00",
    "end_time": "2024-05-04T23:59:59",
    "min_magnitude": 3,
    "limit"

In [20]:
print(response.final_answer["content"])

Here are 10 earthquakes that occurred yesterday with a magnitude greater than 3:

1. [M 4.1 - 92 km SE of Karpathos, Greece](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwdg)
2. [M 4.4 - 9 km SSW of Pahiatua, New Zealand](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwde)
3. [M 4.8 - Kermadec Islands region](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwd2)
4. [M 3.8 - 49 km W of Rincón, Puerto Rico](https://earthquake.usgs.gov/earthquakes/eventpage/pr2024125000)
5. [M 4.6 - off the coast of Ecuador](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwcv)
6. [M 4.3 - off the coast of Oregon](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwcq)
7. [M 4.5 - 37 km S of Finschhafen, Papua New Guinea](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwcn)
8. [M 4.4 - 106 km W of El Aguilar, Argentina](https://earthquake.usgs.gov/earthquakes/eventpage/us6000mwcm)
9. [M 4.5 - 9 km E of Livingstonia, Malawi](https://earthquake.usgs.gov/e

In [21]:
response = earthquake_agent.invoke({"question": "Can MegaQuakes really happen? Like a magnitude 10 or larger?"})

05/05/24 18:27:13 INFO Prompt:
Can MegaQuakes really happen? Like a magnitude 10 or larger?
05/05/24 18:27:26 INFO Raw response:
{
  "thought": "The user is asking about the possibility of a magnitude 10 or larger earthquake, also known as a 'MegaQuake'. According to the Richter Scale, which is used to measure the energy released by earthquakes, a magnitude 10 earthquake has never been recorded. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would release 31.6 times more energy than a magnitude 9.5 earthquake. Theoretically, a magnitude 10 earthquake is possible, but it is highly unlikely because it would require a fault line of over 1000 km in length to rupture all at once. This is larger than any known active fault line on Earth.",
  "tool": "Final Answer",
  "tool_input": {"content": "According to the Richter Scale, which is used to measure the energy released by earthquakes, a magnitude 10 earthquake has never been recorded. The

In [22]:
print(response.final_answer["content"])

According to the Richter Scale, which is used to measure the energy released by earthquakes, a magnitude 10 earthquake has never been recorded. The largest earthquake ever recorded was a magnitude 9.5 in Chile in 1960. A magnitude 10 earthquake would release 31.6 times more energy than a magnitude 9.5 earthquake. Theoretically, a magnitude 10 earthquake is possible, but it is highly unlikely because it would require a fault line of over 1000 km in length to rupture all at once. This is larger than any known active fault line on Earth.


## 4. Service-based AI Agents

In [23]:
import pandas as pd
from sklearn.metrics import accuracy_score

### Sentiment Analysis

In [24]:
df = pd.read_csv("./data/tweets.csv.gz", compression="gzip", encoding="latin-1", names=["sentiment", "id", "date", "query", "user", "tweet"])
df = df.dropna()
df = df.where(df.sentiment != 2)
df["sentiment"] = df["sentiment"].map({4: 1, 0: 0})
df_sampled = df.sample(n=10)
df.head()

,sentiment,id,date,query,user,tweet
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [25]:
system_prompt = """Take the following tweet and determine the sentiment of the review. 

Respond with 1 (positive) or 0 (negative).

If you don't receive a tweet, respond with -1.
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class Output(BaseModel):
    sentiment: int = Field(description="The sentiment of the tweet.")

sentiment_analysis_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt="Tweet:\n{tweet}",
    task_prompt_variables=["tweet"],
    tools=None,
    output_format=Output,
    iterations=2,
)

In [26]:
def classify_sentiment(tweet: str) -> int:
    response = sentiment_analysis_agent.invoke({'tweet': tweet})
    return response.final_answer['sentiment'] or 0

In [27]:
df_sampled["prediction"] = [classify_sentiment(tweet) for tweet in df_sampled.tweet]

05/05/24 18:27:32 INFO Prompt:
Tweet:
the sort of nervous that makes you need a poo 
05/05/24 18:27:35 INFO Raw response:
{
  "thought": "The tweet seems to express a negative sentiment as it talks about feeling nervous.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 0}
}
05/05/24 18:27:35 INFO Thought:
The tweet seems to express a negative sentiment as it talks about feeling nervous.
05/05/24 18:27:35 INFO Final answer:
{'sentiment': 0}
05/05/24 18:27:35 INFO Prompt:
Tweet:
I love it when I am running late &amp; out the door &amp; one of my cats makes a run for it, as if I were a bad mom. 
05/05/24 18:27:39 INFO Raw response:
{
  "thought": "The tweet seems to express frustration or annoyance, which is a negative sentiment.",
  "tool": "Final Answer",
  "tool_input": {"sentiment": 0}
}
05/05/24 18:27:39 INFO Thought:
The tweet seems to express frustration or annoyance, which is a negative sentiment.
05/05/24 18:27:39 INFO Final answer:
{'sentiment': 0}
05/05/24 18:27:39 INF

In [28]:
print(f"Accuracy: {accuracy_score(df_sampled.sentiment, df_sampled.prediction)}")

Accuracy: 0.7


## Multi-Agents

In [29]:
from io import StringIO
from language_models.agents.chain import AgentChain

### Machine Learning Code Generation

In [30]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

Respond with 1 of the following machine learning problems:
- Classification
- Regression
- Clustering
- Time series forecasting"""

task_prompt = """Choose the machine learning problem best suited for the following problem and dataset.

Problem description: 
{problem_description}

Dataset:
Number of rows: {dataset_size}
Schema: 
{dataset_schema}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=128,
    float=0.0,
)

class ModelingProblem(BaseModel):
    modeling_problem: str = Field(description="The machine learning problem.")

problem_finder_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["problem_description", "dataset_size", "dataset_schema"],
    tools=None,
    output_format=ModelingProblem,
    iterations=5,
)

In [31]:
system_prompt = """You are a Data Science agent, which helps the user solve machine learning problems.

You can solve machine learning problems for:
- Classification
- Regression
- Clustering
- Time series forecasting

You have access to the following Python libraries:
- pandas
- numpy
- scikit-learn"""

task_prompt = """Given the following machine learning problem, respond with Python code.

Modeling problem: {modeling_problem}

Dataset:
Number of rows: {dataset_size}
Schema: 
{dataset_schema}
First 10 rows of dataset:
{dataset_snippet}"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=1024,
    float=0.0,
)

class AutoMLCode(BaseModel):
    code: str = Field(description="The Python machine learning code.")

auto_ml_agent = ReActAgent.create(
    llm=llm,
    system_prompt=system_prompt,
    task_prompt=task_prompt,
    task_prompt_variables=["modeling_problem", "dataset_size", "dataset_schema", "dataset_snippet"],
    tools=None,
    output_format=AutoMLCode,
    iterations=10,
)

In [32]:
chain = AgentChain(
    chain=[problem_finder_agent, auto_ml_agent],
    chain_variables=["problem_description", "dataset_size", "dataset_schema", "dataset_snippet"]
)

In [33]:
info_str = StringIO()
df.info(buf=info_str)
dataset_schema = info_str.getvalue()

In [34]:
response = chain.invoke({"problem_description": "I want to classify the sentiment of tweets.", "dataset_size": len(df), "dataset_schema": dataset_schema, "dataset_snippet": str(df.head(10).to_markdown())})

05/05/24 18:28:05 INFO Prompt:
Choose the machine learning problem best suited for the following problem and dataset.

Problem description: 
I want to classify the sentiment of tweets.

Dataset:
Number of rows: 1600000
Schema: 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column     Non-Null Count    Dtype 
---  ------     --------------    ----- 
 0   sentiment  1600000 non-null  int64 
 1   id         1600000 non-null  int64 
 2   date       1600000 non-null  object
 3   query      1600000 non-null  object
 4   user       1600000 non-null  object
 5   tweet      1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB

05/05/24 18:28:10 INFO Raw response:
{
  "thought": "The problem is about classifying sentiments of tweets, which is a typical classification problem in machine learning. The dataset provided also supports this as it contains a 'sentiment' column which can be used as the target v

In [35]:
print(response.final_answer["code"])

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

# Load the data
# df = pd.read_csv('data.csv')

# For demonstration purposes, we will use a subset of the data
# df = df.sample(n=10000, random_state=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['tweet'], df['sentiment'], test_size=0.2, random_state=1)

# Convert the text data into a matrix of token counts
vectorizer = CountVectorizer()
X_train_counts = vectorizer.fit_transform(X_train)
X_test_counts = vectorizer.transform(X_test)

# Train the model
model = LogisticRegression()
model.fit(X_train_counts, y_train)

# Evaluate the model
accuracy = model.score(X_test_counts, y_test)
print(f'Accuracy: {accuracy}')


### Forecasting

In [36]:
from datetime import datetime, timedelta
from language_models.tools.forecasting import get_earthquakes_data, ml_model

In [37]:
class Forecast(BaseModel):
    start_time: str = Field(None, description='Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')
    end_time: str = Field(None, description='Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.')

def forecast(start_time = None, end_time = None):
    if start_time is None:
        start_time = (datetime.now() - timedelta(days=30)).date()
    if end_time is None:
        end_time = (datetime.now().date())
    df = get_earthquakes_data('https://earthquake.usgs.gov/fdsnws/event/1/query?', start_time, end_time)
    df_pred = ml_model.predict(df)
    return {'predictions': df_pred.to_dict(orient='records')}

In [38]:
forecasting_tool = Tool(func=forecast, name='forecast', description='Test forecast model on real-time events.', args_schema=Forecast)

In [39]:
task_prompt = """Take the following question and determine the start and end time to respond with.

Question:
{question}
"""

llm = OpenAILanguageModel(
    proxy_client=proxy_client,
    model='gpt-4',
    max_tokens=256,
    float=0.0,
)

class DateRange(BaseModel):
    start_time: str = Field(description="Limit to events on or after the specified start time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")
    end_time: str = Field(description="Limit to events on or before the specified end time. NOTE: All times use ISO8601 Date/Time format. Unless a timezone is specified, UTC is assumed.")

time_wizard_agent = ReActAgent.create(
    llm=llm,
    system_prompt="",
    task_prompt=task_prompt,
    task_prompt_variables=["question"],
    tools=[current_date_tool],
    output_format=DateRange,
    iterations=5,
)

In [40]:
chain = AgentChain(
    chain=[time_wizard_agent, forecasting_tool], 
    chain_variables=["question"],
)

In [41]:
response = chain.invoke({"question": "Run a forecast using the past week as data."})

05/05/24 18:28:26 INFO Prompt:
Take the following question and determine the start and end time to respond with.

Question:
Run a forecast using the past week as data.

05/05/24 18:28:29 INFO Raw response:
{
  "thought": "To answer this question, I need to determine the current date and then calculate the date one week ago.",
  "tool": "Current Date",
  "tool_input": {}
}
05/05/24 18:28:29 INFO Thought:
To answer this question, I need to determine the current date and then calculate the date one week ago.
05/05/24 18:28:29 INFO Tool:
Current Date
05/05/24 18:28:29 INFO Tool input:
{}
05/05/24 18:28:29 INFO Tool response:
2024-05-05 18:28:29.404260
05/05/24 18:28:35 INFO Raw response:
{
  "thought": "The current date is 2024-05-05. To run a forecast using the past week as data, the start time would be 7 days before the current date and the end time would be the current date.",
  "tool": "Final Answer",
  "tool_input": {
    "start_time": "2024-04-28T18:28:29.404260",
    "end_time": "20

In [42]:
print(response.final_answer['forecast'])

{'predictions': [{'time': Timestamp('2024-04-28 18:34:52.242000+0000', tz='UTC'), 'prediction': 1.998897, 'latitude': 59.4218, 'longitude': -152.5771, 'mag': 1.7, 'id': 'ak0245h3vi14', 'place': '37 km W of Nanwalek, Alaska', 'location': '37 km W of Nanwalek, Alaska'}, {'time': Timestamp('2024-04-28 18:39:57.690000+0000', tz='UTC'), 'prediction': 2.016911, 'latitude': 51.8456666666667, 'longitude': -177.588166666667, 'mag': 0.26, 'id': 'av93108191', 'place': '65 km W of Adak, Alaska', 'location': '65 km W of Adak, Alaska'}, {'time': Timestamp('2024-04-28 18:42:02+0000', tz='UTC'), 'prediction': 2.745448, 'latitude': 32.6798333, 'longitude': -115.8783333, 'mag': 1.12, 'id': 'ci40560679', 'place': '12 km ESE of Ocotillo, CA', 'location': '12 km ESE of Ocotillo, CA'}, {'time': Timestamp('2024-04-28 18:48:41.110000+0000', tz='UTC'), 'prediction': 2.971256, 'latitude': 19.2049999237061, 'longitude': -155.370666503906, 'mag': 1.88, 'id': 'hv74193092', 'place': '11 km E of Pāhala, Hawaii', 'lo